### Import Libraries and Load Model

In [2]:
import sys
sys.path.insert(0, '../')
from aux import *
#%load_ext autoreload
#%autoreload 2
##Clear Memory 
tf.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()
##


if 1: 
    NUM_GPU = len(get_available_gpus())
    if(NUM_GPU>0) :
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

    print(tf.__version__)
    print(keras.__version__)
    #tpu_grpc_url = TPUClusterResolver(tpu=['edv-tpu2'] , zone='us-central1-a').get_master()




    ### Load the Model 
    model_conditions ='Glu' # Glu
    model_conditions_original ='Glu' # Glu


    model , scaler, batch_size = load_model(model_conditions)


W0922 18:25:54.157451 46970166161024 deprecation_wrapper.py:119] From ../aux.py:30: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



1.14.0
2.2.4-tf


/ahg/regevdata/users/edv/software/anaconda3/envs/me/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
W0922 18:26:03.278317 46970166161024 deprecation.py:506] From /ahg/regevdata/users/edv/software/anaconda3/envs/me/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0922 18:26:05.008341 46970166161024 deprecation.py:506] From /ahg/regevdata/users/edv/software/anaconda3/envs/me/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecat

### Figure Specific Functions and Imports


In [3]:
np.random.seed(0)


#########################################################################################################
#########################################################################################################
### Generate all possible single mutations in population : population_next 

def population_mutator( population_current , args) :
    population_current = population_remove_flank(population_current)
    population_next = []  
    for i in range(len(population_current)) :         
        for j in range(args['sequence_length']) : 
        #First create three copies of the same individual, one for each possible mutation at the basepair.
            population_next.append(list(population_current[i]))
            population_next.append(list(population_current[i]))
            population_next.append(list(population_current[i]))
            
            if (population_current[i][j] == 'A') :
                population_next[3*(args['sequence_length']*i + j) ][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
                
            elif (population_current[i][j] == 'C') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'A'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
            
            elif (population_current[i][j] == 'G') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'A'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'T'
                
            elif (population_current[i][j] == 'T') :
                population_next[3*(args['sequence_length']*i + j)][j] = 'C'
                population_next[3*(args['sequence_length']*i + j) + 1][j] = 'G'
                population_next[3*(args['sequence_length']*i + j) + 2][j] = 'A'
             
        
    population_next= population_add_flank(population_next)        
    return list(population_next)



#########################################################################################################
#########################################################################################################
#########################################################################################################




#########################################################################################################
#########################################################################################################
### Find the single bp mutation that has highest expression 
def maximize_next_generation(population_current, population_current_fitness): 
    population_next_all = population_mutator(list(population_current) , args)
    population_next_all_predictions = evaluate_model(list(population_next_all),model,scaler,batch_size)
    
    population_next_max_seq = list(population_current)  
    population_next_max_seq_fitness = list(population_current_fitness)
    for i in tqdm(range(len(population_current))) :  
        max_score = -np.inf
        for j in range(3*args['sequence_length']) : 
            if (population_next_all_predictions[3*args['sequence_length']*i + j] > max_score) :
                population_next_max_seq[i] = population_next_all[3*args['sequence_length']*i + j]
                population_next_max_seq_fitness[i] = population_next_all_predictions[3*args['sequence_length']*i + j]
                max_score = population_next_all_predictions[3*args['sequence_length']*i + j]
  
    return list(population_next_max_seq) , list(population_next_max_seq_fitness)
    #print population_next_all_predictions
#########################################################################################################
#########################################################################################################
#########################################################################################################





#########################################################################################################
#########################################################################################################
### INTRA FUNCTION NAMES ARE WRONG, VARIABLE IS RIGHT THOUGH , Find the single bp mutation that has least expression 
def minimize_next_generation(population_current, population_current_fitness): 
    population_next_all = population_mutator(list(population_current) , args)
    population_next_all_predictions = evaluate_model(list(population_next_all),model,scaler,batch_size)
    population_next_min_seq = list(population_current)  
    population_next_min_seq_fitness = list(population_current_fitness)
    for i in range(len(population_current)) :  
        min_score = np.inf
        for j in range(3*args['sequence_length']) : 
            if (population_next_all_predictions[3*args['sequence_length']*i + j] < min_score) :
                population_next_min_seq[i] = population_next_all[3*args['sequence_length']*i + j]
                population_next_min_seq_fitness[i] = population_next_all_predictions[3*args['sequence_length']*i + j]
                min_score = population_next_all_predictions[3*args['sequence_length']*i + j]
  
    return list(population_next_min_seq) , list(population_next_min_seq_fitness)
    #print population_next_all_predictions
#########################################################################################################
#########################################################################################################
#########################################################################################################



#########################################################################################################
#########################################################################################################
### Select Randomly from the 240 possible mutations
def neutral_next_generation(population_current, population_current_fitness): 
    population_next_all = population_mutator(list(population_current) , args)
    population_next_neutral_seq = list(population_current)  
    for i in tqdm(range(len(population_current))) : 
        j = np.random.choice(3*args['sequence_length'])    ### Pick a random but different neighbour     
        population_next_neutral_seq[i] = population_next_all[3*args['sequence_length']*i + j]

    population_next_neutral_seq_fitness = evaluate_model(list(population_next_neutral_seq),model,scaler,batch_size)


    return list(population_next_neutral_seq) , list(population_next_neutral_seq_fitness)
    #print population_next_all_predictions
#########################################################################################################
#########################################################################################################
#########################################################################################################


#########################################################################################################
#########################################################################################################
#########################################################################################################
### Plotting Functions

def _statsmodels_univariate_kde(data, kernel, bw, gridsize, cut, clip,
                                cumulative=False):
    """Compute a univariate kernel density estimate using statsmodels."""
    fft = kernel == "gau"
    kde = smnp.KDEUnivariate(data)
    kde.fit(kernel, bw, fft, gridsize=gridsize, cut=cut, clip=clip)
    if cumulative:
        grid, y = kde.support, kde.cdf
    else:
        grid, y = kde.support, kde.density
    return grid, y


def _scipy_univariate_kde(data, bw, gridsize, cut, clip):
    """Compute a univariate kernel density estimate using scipy."""
    try:
        kde = stats.gaussian_kde(data, bw_method=bw)
    except TypeError:
        kde = stats.gaussian_kde(data)
        if bw != "scott":  # scipy default
            msg = ("Ignoring bandwidth choice, "
                   "please upgrade scipy to use a different bandwidth.")
            warnings.warn(msg, UserWarning)
    if isinstance(bw, string_types):
        bw = "scotts" if bw == "scott" else bw
        bw = getattr(kde, "%s_factor" % bw)() * np.std(data)
    grid = _kde_support(data, bw, gridsize, cut, clip)
    y = kde(grid)
    return grid, y

#########################################################################################################
#########################################################################################################
#########################################################################################################
%matplotlib inline

### Prepare Starting sequences 

In [6]:
### Native Sequences
native_sequences_df = pd.read_csv(os.path.join('..','..','data','native_sequences_only','nativeChunks.2.uniq.txt') , header=None)


#pd.read_csv(os.path.join('..','..','data','native_sequences_only','native_sequences.txt') , sep='\t', index_col=0, header=None)
native_sequences_list = population_add_flank(list(native_sequences_df[0]))#list(native_sequences_df[1])


args  = { 'population_size' : len(native_sequences_list ), 
         'sequence_length' : 80 , 'nucleotide_frequency' :[0.25,0.25,0.25,0.25] , 'randomizer' : np.random } 


random = 1 ### If you wanna show the neutral trajectory on random sequences 


In [7]:
### Random Sequences
if 0 :
    if random==1 :
        random_sequences_list = population_generator(args)

### Trajectories under Random Genetic Drift

In [8]:
if 1: 
    if random==1  :
        ### Generate Data
        generation_neutral_seq = []
        generation_neutral_seq_fitness = []

        population = random_sequences_list #idt_seqs#native_mutate_sequences###population_generator( args ) ## replace with new sequence list

        generation_neutral_seq.append(list(population)) #population to maximize , generation_max_seq[0]
        generation_neutral_seq_fitness.append(list(evaluate_model(list(population),model,scaler,batch_size)))





        args['num_rounds'] = 40
        for i in tqdm(range(args['num_rounds'])) : 
            population_next_neutral_seq , population_next_neutral_seq_fitness = neutral_next_generation(generation_neutral_seq[i] , generation_neutral_seq_fitness[i])
            generation_neutral_seq.append(population_next_neutral_seq)
            generation_neutral_seq_fitness.append(population_next_neutral_seq_fitness)


        len(generation_neutral_seq_fitness)


        neutral_array = np.transpose(np.asarray(generation_neutral_seq_fitness))
        neutral_dataframe = pd.DataFrame(data = neutral_array , index = generation_neutral_seq[0])



        neutral_dataframe.to_csv(model_conditions+'neutral_evolution_random.tsv' ,sep='\t')










    else :
        ### Generate Data
        generation_neutral_seq = []
        generation_neutral_seq_fitness = []

        population = native_sequences_list #idt_seqs#native_mutate_sequences###population_generator( args ) ## replace with new sequence list

        generation_neutral_seq.append(list(population)) #population to maximize , generation_max_seq[0]
        generation_neutral_seq_fitness.append(list(evaluate_model(list(population),model,scaler,batch_size)))





        args['num_rounds'] = 40
        for i in tqdm(range(args['num_rounds'])) : 
            population_next_neutral_seq , population_next_neutral_seq_fitness = neutral_next_generation(generation_neutral_seq[i] , generation_neutral_seq_fitness[i])
            generation_neutral_seq.append(population_next_neutral_seq)
            generation_neutral_seq_fitness.append(population_next_neutral_seq_fitness)


        len(generation_neutral_seq_fitness)


        neutral_array = np.transpose(np.asarray(generation_neutral_seq_fitness))
        neutral_dataframe = pd.DataFrame(data = neutral_array , index = generation_neutral_seq[0])



        neutral_dataframe.to_csv(model_conditions+'neutral_evolution_native.tsv' ,sep='\t')

### Trajectories under SSWM

In [12]:
r_n = "native"
if 0 : 
    model , scaler, batch_size = load_model(model_conditions_original)

In [14]:
if 1 :
    #neutral_dataframe = pd.read_csv('neutral_evolution.tsv' , sep='\t' , index_col=0)
    if r_n == "native" : 
        population = native_sequences_list#neutral_dataframe.index.values[0:10000] #idt_seqs####population_generator( args ) ## replace with new sequence list
    elif r_n == "random" : 
        population = neutral_dataframe.index.values[0:10000] #idt_seqs####population_generator( args ) ## replace with new sequence list




    generation_max_seq = []
    generation_max_seq_fitness = []

    generation_min_seq = []
    generation_min_seq_fitness = []


    generation_max_seq.append(list(population)) #population to maximize , generation_max_seq[0]
    generation_max_seq_fitness.append(list(evaluate_model(list(population),model,scaler,batch_size)))

    generation_min_seq.append(list(population))
    generation_min_seq_fitness.append(list(generation_max_seq_fitness[0]))







    args['num_rounds'] = 10
    for i in tqdm(range(args['num_rounds'])) : 
        population_next_max_seq , population_next_max_seq_fitness = maximize_next_generation(generation_max_seq[i] , generation_max_seq_fitness[i])
        generation_max_seq.append(population_next_max_seq)
        generation_max_seq_fitness.append(population_next_max_seq_fitness)

        population_next_min_seq , population_next_min_seq_fitness = minimize_next_generation(generation_min_seq[i] , generation_min_seq_fitness[i])
        generation_min_seq.append(population_next_min_seq)
        generation_min_seq_fitness.append(population_next_min_seq_fitness)



    len(generation_max_seq_fitness)


    max_array = np.transpose(np.asarray(generation_max_seq_fitness))
    max_dataframe = pd.DataFrame(data = max_array , index = generation_max_seq[0])

    min_array = np.transpose(np.asarray(generation_min_seq_fitness))
    min_dataframe = pd.DataFrame(data = min_array , index = generation_min_seq[0])

    max_melt = max_dataframe.melt(value_name='expression' , var_name='edit_distance')
    min_melt = min_dataframe.melt(value_name='expression' , var_name='edit_distance')

    max_melt['Selection Direction'] = 'max'
    min_melt['Selection Direction'] = 'min'

    merged_melt = min_melt.append(max_melt)


    max_seq_array = np.transpose(np.asarray(generation_max_seq))
    max_seq_dataframe = pd.DataFrame(data = max_seq_array , index = generation_max_seq[0])

    min_seq_array = np.transpose(np.asarray(generation_min_seq))
    min_seq_dataframe = pd.DataFrame(data = min_seq_array , index = generation_min_seq[0])



In [16]:
### Save/Load Data Frames 
if 0: 
    max_seq_dataframe.to_csv(model_conditions+r_n+'_'+'directed_evolution_max_seq.tsv' ,sep='\t')
    min_seq_dataframe.to_csv(model_conditions+r_n+'_'+'directed_evolution_min_seq.tsv' ,sep='\t')

    max_dataframe.to_csv(model_conditions+r_n+'_'+'directed_evolution_max.tsv' ,sep='\t')
    min_dataframe.to_csv(model_conditions+r_n+'_'+'directed_evolution_min.tsv' ,sep='\t')

else : 
    max_dataframe = pd.read_csv(model_conditions+r_n+'_'+'directed_evolution_max.tsv' ,sep='\t' , index_col=0)
    min_dataframe = pd.read_csv(model_conditions+r_n+'_'+'directed_evolution_min.tsv' ,sep='\t', index_col=0)

max_melt = max_dataframe.melt(value_name='expression' , var_name='edit_distance')
min_melt = min_dataframe.melt(value_name='expression' , var_name='edit_distance')

max_melt['Selection Direction'] = 'max'
min_melt['Selection Direction'] = 'min'

merged_melt = min_melt.append(max_melt)



### Trajectories for optimizing conflicting expression objectives

In [17]:
r_n = "native" 

In [18]:
### Load the two models into model1,scaler1 and model2,scaler2

model_conditions_original=='SC_Ura'

if 0 : 
    ### Load the other model
    if model_conditions_original=='SC_Ura':
        model1 , scaler1, batch_size = load_model('SC_Ura')
        model2 , scaler2, batch_size = load_model('Glu')

    else :
        model1 , scaler1, batch_size = load_model('Glu')
        model2 , scaler2, batch_size = load_model('SC_Ura')

    multiobjective_model = Model([model1.input,model2.input], Concatenate(axis=1)([model1.output,model2.output]))


#neutral_dataframe = pd.read_csv('neutral_evolution.tsv' , sep='\t' , index_col=0)
if r_n == "native" : 
    population = native_sequences_list#neutral_dataframe.index.values[0:10000] #idt_seqs####population_generator( args ) ## replace with new sequence list
elif r_n == "random" : 
    population = neutral_dataframe.index.values[0:10000] #idt_seqs####population_generator( args ) ## replace with new sequence list



In [19]:
if 1 :
    def evaluate_multiobjective_model(X,multiobjective_model, batch_size) :
        X = seq2feature(X)
        Y_pred = multiobjective_model.predict( [X,X] , batch_size = batch_size , verbose=1)
        return scaler1.inverse_transform(Y_pred[:,0]) , scaler2.inverse_transform(Y_pred[:,1])



    #########################################################################################################
    #########################################################################################################
    ### Find the single bp mutation that has highest expression 
    def mo_maximize_next_generation(population_current, population_current_fitness): 
        population_next_all = population_mutator(list(population_current) , args)
        model_predicitons = evaluate_multiobjective_model(population_next_all,multiobjective_model,batch_size)
        population_next_all_predictions = list(model_predicitons[0] -  model_predicitons[1])

        population_next_max_seq = list(population_current)  
        population_next_max_seq_fitness = list(population_current_fitness)
        for i in tqdm(range(len(population_current))) :  
            max_score = -np.inf
            for j in range(3*args['sequence_length']) : 
                if (population_next_all_predictions[3*args['sequence_length']*i + j] > max_score) :
                    population_next_max_seq[i] = population_next_all[3*args['sequence_length']*i + j]
                    population_next_max_seq_fitness[i] = population_next_all_predictions[3*args['sequence_length']*i + j]
                    max_score = population_next_all_predictions[3*args['sequence_length']*i + j]

        return list(population_next_max_seq) , list(population_next_max_seq_fitness)
        #print population_next_all_predictions
    #########################################################################################################
    #########################################################################################################
    #########################################################################################################





    #########################################################################################################
    #########################################################################################################
    ### INTRA FUNCTION NAMES ARE WRONG, VARIABLE IS RIGHT THOUGH , Find the single bp mutation that has least expression 
    def mo_minimize_next_generation(population_current, population_current_fitness): 
        population_next_all = population_mutator(list(population_current) , args)
        model_predicitons = evaluate_multiobjective_model(population_next_all,multiobjective_model,batch_size)
        population_next_all_predictions = list(model_predicitons[0] -  model_predicitons[1])

        population_next_min_seq = list(population_current)  
        population_next_min_seq_fitness = list(population_current_fitness)
        for i in range(len(population_current)) :  
            min_score = np.inf
            for j in range(3*args['sequence_length']) : 
                if (population_next_all_predictions[3*args['sequence_length']*i + j] < min_score) :
                    population_next_min_seq[i] = population_next_all[3*args['sequence_length']*i + j]
                    population_next_min_seq_fitness[i] = population_next_all_predictions[3*args['sequence_length']*i + j]
                    min_score = population_next_all_predictions[3*args['sequence_length']*i + j]

        return list(population_next_min_seq) , list(population_next_min_seq_fitness)
        #print population_next_all_predictions
    #########################################################################################################
    #########################################################################################################
    #########################################################################################################





    generation_max_seq = []
    generation_max_seq_fitness = []

    generation_min_seq = []
    generation_min_seq_fitness = []


    generation_max_seq.append(list(population)) #population to maximize , generation_max_seq[0]
    model_predicitons = evaluate_multiobjective_model(population,multiobjective_model,batch_size)    
    generation_max_seq_fitness.append(list( model_predicitons[0] -  model_predicitons[1]))

    generation_min_seq.append(list(population))
    generation_min_seq_fitness.append(list(generation_max_seq_fitness[0]))







    args['num_rounds'] = 10
    for i in tqdm(range(args['num_rounds'])) : 
        population_next_max_seq , population_next_max_seq_fitness = mo_maximize_next_generation(generation_max_seq[i] , generation_max_seq_fitness[i])
        generation_max_seq.append(population_next_max_seq)
        generation_max_seq_fitness.append(population_next_max_seq_fitness)

        population_next_min_seq , population_next_min_seq_fitness = mo_minimize_next_generation(generation_min_seq[i] , generation_min_seq_fitness[i])
        generation_min_seq.append(population_next_min_seq)
        generation_min_seq_fitness.append(population_next_min_seq_fitness)



    len(generation_max_seq_fitness)




    max_array = np.transpose(np.asarray(generation_max_seq_fitness))
    max_dataframe = pd.DataFrame(data = max_array , index = generation_max_seq[0])

    min_array = np.transpose(np.asarray(generation_min_seq_fitness))
    min_dataframe = pd.DataFrame(data = min_array , index = generation_min_seq[0])

    max_melt = max_dataframe.melt(value_name='expression' , var_name='edit_distance')
    min_melt = min_dataframe.melt(value_name='expression' , var_name='edit_distance')

    max_melt['Selection Direction'] = 'max'
    min_melt['Selection Direction'] = 'min'

    merged_melt = min_melt.append(max_melt)

    #max_dataframe.to_csv('multiobjective_directed_evolution_max.tsv' ,sep='\t')
    #min_dataframe.to_csv('multiobjective_directed_evolution_min.tsv' ,sep='\t')
    
    
    
    max_seq_array = np.transpose(np.asarray(generation_max_seq))
    max_seq_dataframe = pd.DataFrame(data = max_seq_array , index = generation_max_seq[0])

    min_seq_array = np.transpose(np.asarray(generation_min_seq))
    min_seq_dataframe = pd.DataFrame(data = min_seq_array , index = generation_min_seq[0])


    scura_max_dataframe=copy.deepcopy(max_dataframe)
    scura_min_dataframe=copy.deepcopy(min_dataframe)

    glu_max_dataframe=copy.deepcopy(max_dataframe)
    glu_min_dataframe=copy.deepcopy(min_dataframe)


    for i in range(scura_max_dataframe.shape[1]) : 
        scura_max_dataframe.iloc[:,i] = evaluate_model(max_seq_dataframe.iloc[:,i].values , 
                                                       model1 , scaler1,batch_size)
        scura_min_dataframe.iloc[:,i] = evaluate_model(min_seq_dataframe.iloc[:,i].values , 
                                                       model1 , scaler1,batch_size)


        glu_max_dataframe.iloc[:,i] = evaluate_model(max_seq_dataframe.iloc[:,i].values , 
                                                       model2 , scaler2,batch_size)
        glu_min_dataframe.iloc[:,i] = evaluate_model(min_seq_dataframe.iloc[:,i].values , 
                                                       model2 , scaler2,batch_size)


In [20]:
### Save all the data to TSV / load it
if 1 :
    max_seq_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_max_seq.tsv' , sep='\t')
    min_seq_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_min_seq.tsv' , sep='\t')
    scura_max_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_scura_max_dataframe.tsv' , sep='\t')
    scura_min_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_scura_min_dataframe.tsv' , sep='\t')
    glu_max_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_glu_max_dataframe.tsv' , sep='\t')
    glu_min_dataframe.to_csv(r_n+'_'+'multiobjective_directed_evolution_glu_min_dataframe.tsv' , sep='\t')

    

if r_n == 'native' : 
    scura_max_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_scura_max_dataframe.tsv' , sep='\t',index_col=0)
    scura_min_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_scura_min_dataframe.tsv' , sep='\t',index_col=0)

    glu_max_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_glu_max_dataframe.tsv' , sep='\t',index_col=0)
    glu_min_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_glu_min_dataframe.tsv' , sep='\t',index_col=0)

elif r_n == 'random' : 
    scura_max_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_scura_max_dataframe.tsv' , sep='\t',index_col=0)
    scura_min_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_scura_min_dataframe.tsv' , sep='\t',index_col=0)

    glu_max_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_glu_max_dataframe.tsv' , sep='\t',index_col=0)
    glu_min_dataframe = pd.read_csv(r_n+'_'+'multiobjective_directed_evolution_glu_min_dataframe.tsv' , sep='\t',index_col=0)


